In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=0.5, std=0.5),
                              ])

trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

for images, labels in trainloader:
    print(type(images))
    print(images.shape)
    print(labels.shape)
    break

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [3]:
model =  nn.Sequential(nn.Linear(784, 128),
                       nn.ReLU(),
                       nn.Linear(128, 64),
                       nn.ReLU(),
                       nn.Linear(64,10),
                       nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

In [4]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
epochs=10

In [6]:
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else :
        print(f"training loss: {running_loss/len(trainloader)}")

training loss: 1.0174516996722232
training loss: 0.3810220019840228
training loss: 0.32360381720416836
training loss: 0.292163591267966
training loss: 0.26808902769804255
training loss: 0.24736028981170674
training loss: 0.22805662323702883
training loss: 0.21048990285584032
training loss: 0.1949429207963984
training loss: 0.18045274362063357


In [8]:
import helper

images, labels = next(iter(trainloader))

img = images[0].view(1 ,784)
with torch.no_grad():
    logits = model.forward(img)
    
ps = softmax(logits, dim=1)
helper.view_classify(img.view(1,28,28), ps)

ModuleNotFoundError: No module named 'helper'